# RAG System
Imagine you're asking a super smart friend (that's the Large Language Model or LLM) a question. A RAG system is like giving your super smart friend a quick way to look things up before answering.
How RAG works:

1.  **Documents are Chunked and Embedded:** Your knowledge base is broken into small pieces, and each piece is converted into a numerical "meaning" representation.
2.  **Embeddings are Stored:** These numerical representations are then saved in a special database designed for quick similarity searches.
3.  **User Submits a Query:** You ask your question to the RAG system.
4.  **Query is Embedded:** Your question is also converted into a numerical "meaning" representation.
5.  **Relevant Chunks are Retrieved:** The system searches its database for document chunks whose meanings are most similar to your query's meaning.
6.  **Context is Formed:** The retrieved relevant text chunks are then added to your original query, creating an enriched prompt.
7.  **LLM Generates Answer:** A Large Language Model uses this enriched prompt to provide a factual and comprehensive response.

### RAG application built on gemini 

In [3]:
 #install required packages
! uv pip install langchain langchain-community langchain-google-genai python-dotenv streamlit langchain-experimental sentence-transformers langchain-chroma langchainhub pypdf rapidocr-onnxruntime
! uv pip install faiss-cpu


Using Python 3.11.4 environment at: C:\Users\97798\Desktop\myproject\.venv
Resolved 151 packages in 41.02s
   Building pypika==0.48.9
      Built pypika==0.48.9
Prepared 98 packages in 6m 14s
Installed 98 packages in 1m 23s
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.12.13
 + aiosignal==1.3.2
 + altair==5.5.0
 + attrs==25.3.0
 + backoff==1.11.1
 + bcrypt==4.3.0
 + blinker==1.9.0
 + build==1.2.2.post1
 + chromadb==1.0.13
 + click==8.2.1
 + coloredlogs==15.0.1
 + dataclasses-json==0.6.7
 + distro==1.9.0
 + durationpy==0.10
 + filelock==3.18.0
 + flatbuffers==25.2.10
 + frozenlist==1.7.0
 + fsspec==2025.5.1
 + gitdb==4.0.12
 + gitpython==3.1.44
 + httptools==0.6.4
 + httpx-sse==0.4.1
 + huggingface-hub==0.33.1
 + humanfriendly==10.0
 + importlib-metadata==8.7.0
 + importlib-resources==6.5.2
 + jinja2==3.1.6
 + joblib==1.5.1
 + jsonschema==4.24.0
 + jsonschema-specifications==2025.4.1
 + kubernetes==33.1.0
 + langchain-chroma==0.2.4
 + langchain-community==0.3.26
 + langchain-experimental==0.

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mypdf.pdf")
data = loader.load()  
#data

ValueError: File path mypdf.pdf is not a valid file or url

In [ ]:
len(data)

14

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  42


In [13]:
docs[7]

Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-06-28T03:44:31+00:00', 'moddate': '2025-06-28T03:44:31+00:00', 'source': 'mypdf.pdf', 'total_pages': 14, 'page': 2, 'page_label': '3'}, page_content='(IOE) \n4 Years 96 \nBachelor in \nComputer \nEngineering \nBCT Institute of \nEngineering \n(IOE) \n4 Years 48 \nBachelor in \nElectronics, \nCommunication \nand Information \nEngineering \nBEI Institute of \nEngineering \n(IOE) \n4 Years 48 \nBachelor in \nArchitecture \nB.Arch Institute of \nEngineering \n(IOE) \n5 Years 48 \nBachelor of \nScience in \nComputer \nScience and \nInformation \nTechnology \nBSc.CSIT Institute of \nScience and \nTechnology \n(IOST) \n4 Years 48 \nBachelor of BCA Faculty of 4 Years 36')

### Get an API key: 
Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/


**A vector database** stores data as numerical "vectors" to enable semantic search, finding items based on meaning rather than keywords.   
**ChromaDB** is an easy-to-use, open-source vector database that simplifies storing and querying these embeddings, with built-in persistence for your data.   
**FAISS** is a high-performance library for efficient similarity search and clustering of dense vectors, widely used for building scalable vector search applications.

In [23]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv() 


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [24]:
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)
# vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

The `retriever` enables efficient semantic search by retrieving the most relevant document chunks from the vectorstore based on the user's query.

In [39]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("Location of Himalayan Engineering College?")


In [40]:
len(retrieved_docs)

10

In [41]:
print(retrieved_docs[5].page_content)

Question: Where is HCOE located, and has its location changed? 
Answer: HCOE is currently located at Chyasal, Lalitpur, Nepal.2 The college moved to this 
location before June 2012. Previously, it was situated in Shankhamul and Buddhanagar in 
Kathmandu.4 
Question: What is the primary affiliation of HCOE? 
Answer: Himalaya College of Engineering is affiliated with Tribhuvan University (TU), Nepal. Its 
engineering and architecture programs are specifically affiliated with the Institute of 
Engineering (IOE), TU.1 
Question: Who are the key administrative leaders at HCOE? 
Answer: The Principal of Himalaya College of Engineering is Mr. Kishor Gautam.7 The college 
also has an Administrative Director who is a key figure in its management.1 Er. Kamal Sapkota 
serves as the Deputy Head of the Civil Engineering Department.9 
Question: What is the stated vision of HCOE? 
Answer: The vision of HCOE is to establish itself as one of the major centers of learning in the


### Langchain 
LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides tools for connecting LLMs to external data sources, enabling retrieval-augmented generation, and building advanced conversational and reasoning workflows.

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.3, max_tokens=500)

**Prompt engineering** is the practice of crafting effective inputs (prompts) to guide large language models (LLMs) toward producing accurate, relevant, and useful outputs.  
It involves experimenting with wording, structure, and context to optimize model responses for specific tasks or applications.  
Good prompt engineering can significantly improve the quality and reliability of LLM-powered solutions.

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "who is the principle of Himalayan Engineering College?"})
print(response["answer"])

The principal of Himalaya College of Engineering is Mr. Kishor Gautam.
